# Exploring `hvplot`
PyViz released a library called `hvplot` as a high-level plotting API for the PyData ecosystem.
This is a reproduction of the example in this [blogpost](http://blog.pyviz.org/hvplot_announcement.html#Support-for-geographic-plots) and to see whether it can be extended with other `bokeh` tools such as [`FreehandDraw`](https://bokeh.pydata.org/en/latest/docs/user_guide/tools.html#freehanddrawtool).

## Demo

In [ ]:
import hvplot as hplt
import xarray as xr
import hvplot.xarray
import geoviews as gv
import cartopy.crs as ccrs
import cartopy.feature as cf

Load a dataset using `xarray`

In [ ]:
air_ds = xr.tutorial.open_dataset('air_temperature').load()
air_ds

Define all the elements of our plot

In [ ]:
coastlines = gv.feature.coastline
proj = ccrs.Orthographic(-90, 30)
air_temp = air_ds.air.isel(time=slice(0, 9, 3))
air_plot = air_temp.hvplot.quadmesh(
    'lon', 'lat', projection=proj, project=True, global_extent=True,
    width=600, height=540, cmap='viridis', rasterize=True, dynamic=False)

Combine and display our plot elements

In [ ]:
air_plot * coastlines

Cool, that works as expected.

### `bokeh` vs `matplotlib`

Compare that to plotting with a `matplotlib` backend

In [ ]:
import holoviews as hv

hv.extension('matplotlib')

air_plot_mpl = air_temp.hvplot.quadmesh(
    'lon', 'lat', projection=proj, project=True, global_extent=True,
    width=600, height=540, cmap='viridis', rasterize=True, dynamic=False)

air_plot_mpl * coastlines

Compare **that** to plotting with just `matplotlib`

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection=ccrs.Orthographic(-90, 30))
ax.set_global()
air_temp.isel(time=0).plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree())
ax.coastlines()

## Extending `hvplot` example

A `FreehandDraw` tool must be linked to another element, in this case a `Path`

In [ ]:
import geoviews as gv
from holoviews.streams import FreehandDraw

hv.extension('bokeh')

draw_tool = gv.Path([]).opts(color='magenta', line_width=4)
freehand_stream = FreehandDraw(source=draw_tool)

Let's combine those and test it out

In [ ]:
air_plot * coastlines * draw_tool

Yasssss, it works! We can draw lines on our plotted data.

## Q: Can we now get hold of the scribbles we have made?

In [ ]:
freehand_stream.element

In [ ]:
print(freehand_stream.element)

In [ ]:
freehand_stream.data

In [ ]:
draw_tool.data

Yes we can! From the tool object itself (`freehand_stream`) we can get the information of a scribble back as a `HoloViews` element or the actual `(x,y)` data in lat and lon i.e. georeferencing for free!

## Q: Can we add multiple pens for the forecasters to use?

In [ ]:
cold_front = gv.Path([]).opts(color='blue', line_width=4)
cold_front_pen = FreehandDraw(source=cold_front)

warm_front = gv.Path([]).opts(color='red', line_width=4)
warm_front_pen = FreehandDraw(source=warm_front)

In [ ]:
air_plot * coastlines * warm_front * cold_front

In [ ]:
warm_front_pen.element

In [ ]:
cold_front_pen.element

In [ ]:
warm_front_pen.data

Yep, multiple instances of the same tool (`FreehandDraw`) can be linked to different `Path`s, each with different properties.

**Development goals:**
1. Store more useful metadata with the path
2. Change the icon for each brush. Just color to start with but then more useful symbols.

## Q: Can we add a warning area polyon tool?

In [ ]:
warning_orange = gv.Polygons([]).opts(line_color='orange', line_width=4, 
                                      fill_color='orange', fill_alpha=0.6)
warning_orange_tool = FreehandDraw(source=warning_orange)

In [ ]:
air_plot * coastlines * warning_orange

In [ ]:
warning_orange_tool.element

Polygons can be linked to the same freehand drawing tool. This will be useful for annotating with areas e.g. weather warnings.

## Q: Can we reapply elements to a new plot?

In [ ]:
warning_01 = warning_orange_tool.element

In [ ]:
air_temp_all = air_ds.air.isel(time=slice(0, 9, 1))
air_plot_all = air_temp_all.hvplot.quadmesh(
    'lon', 'lat', projection=proj, project=True, global_extent=True,
    width=600, height=540, cmap='fire', rasterize=True, dynamic=False)

In [ ]:
air_plot_all * coastlines * warning_01

Yes reapplication can be done. This will be useful for sharing and re-using annotation elements.

## Q: Can an element be reapplied and edited?

In [ ]:
warning_02 = warning_orange_tool.element.opts(line_color='orange', line_width=1,
                                              fill_color='orange', fill_alpha=0.4)

In [ ]:
from holoviews import streams

In [ ]:
poly_edit = streams.PolyEdit(source=warning_02)

In [ ]:
air_plot * coastlines * warning_02

In [ ]:
from holoviews.streams import PolyEdit

cold_front_01 = cold_front_pen.element.opts(line_color='blue', line_width=2)

cold_front_edit = PolyEdit(source=cold_front_01)

cold_front_01

In [ ]:
air_plot * coastlines * cold_front_01

Elements can be reallied and edited. This includes restyling the element and linking it to the `PolyEdit` tool.

## Q: Can interactive plot be launched in a `Sidecar`?

In [ ]:
from sidecar import Sidecar

with Sidecar():
    display(air_plot * coastlines * warm_front * cold_front * warning_orange)

## Q: How to add an `Annotable` button?

This will be a button on the `bokeh` toolbar which launches current plot in Annotable webapp, which can be ran in a browser on a touchscreen device.